In [1]:
import cv2
import numpy as np
from PIL import Image

In [2]:
def imshow(a):
    a = a.clip(0, 255).astype("uint8")
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(a))

In [3]:
def load_video(path):
    cap = cv2.VideoCapture(path)
    if cap.isOpened():
        print('Opened the file successfully.')

    print(f"File name: {path}")
    # Height and width of the video
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

    print(f"Height: {height}, Width: {width}")

    # Number of frames in the video
    FPS = cap.get(cv2.CAP_PROP_FPS)
    print(f"FPS: {FPS}")

    return cap, int(height), int(width), FPS

In [4]:
def estimate_piece_color(img, circle):
    # Crop the piece image from the original image
    x, y, r = circle
    piece_img = img[y - r : y + r, x - r : x + r]

    # Calculate the average brightness of the piece
    avg_brightness = np.mean(piece_img)

    # If the average brightness is greater than 150, the piece is white
    if avg_brightness > 64:
        return "white"
    else:
        return "black"

In [5]:
input_file_path = "videos/EASY.mp4"
output_file_path = "test.avi"

test_video, test_height, test_width, test_FPS = load_video(input_file_path)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"DIVX")
test_red_clip = cv2.VideoWriter(
    output_file_path, 
    fourcc, 
    test_FPS, 
    (test_width, test_height),
    )

frame_number = 0
while test_video.isOpened():
    ret, frame = test_video.read()
    frame_number += 1
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        print(frame_number)
        break  # Exit the loop when no frames are retrieved
    

    # Piece detection
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve circle detection
    frame_blur = cv2.GaussianBlur(frame_gray, (5, 5), 0)

    # Use HoughCircles to detect circles
    circles = cv2.HoughCircles(
        frame_blur,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=50,
        param1=50,
        param2=20,
        minRadius=20,
        maxRadius=30, # changet to 35 for more false positives (30 for less)
    )

    if circles is not None:
        # Convert the circle parameters a, b and r to integers
        circles = np.uint16(np.around(circles))

        # Draw the circles on the original image
        for a, b, r in circles[0, :]:
            # Estimate the piece color
            color = estimate_piece_color(frame_blur, (a, b, r))

            # Draw the circle in the output video
            if color == "black":
                cv2.circle(frame, (a, b), r, (0, 0, 0), 3)
            else:
                cv2.circle(frame, (a, b), r, (255, 255, 255), 3)


    # Board detection
    frame_th = cv2.adaptiveThreshold(
        frame_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 10
        )
    frame_edges = cv2.Canny(frame_th, 50, 150)

    # Find HoughLines
    lines = cv2.HoughLines(frame_edges, 1, np.pi / 180, 180)

    # Negative mask of the lines
    mask = np.zeros(frame_edges.shape[:2], dtype=np.uint8)

    # Draw the lines on the mask
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)

        x0 = a * rho
        y0 = b * rho

        x1 = int(x0 + 1000 * -b)
        y1 = int(y0 + 1000 * a)

        x2 = int(x0 - 1000 * -b)
        y2 = int(y0 - 1000 * a)

        cv2.line(mask, (x1, y1), (x2, y2), (255, 255, 255), 1)

    # Find the contours in the mask
    contours, hierarchy = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )

    # Return squares
    squares = []
    areas = []
    for cnt in contours[1:]:
        # Approximate the contour to a polygon
        epsilon = 0.1 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)

        # Check if the polygon has 4 sides
        if len(approx) == 4:
            # Use minimum area rectangle to consider rotation
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)

            # Calculate width and height of the rectangle
            width = np.linalg.norm(box[0] - box[1])
            height = np.linalg.norm(box[1] - box[2])

            aspectRatio = max(width, height) / min(width, height)
            if 0.7 <= aspectRatio <= 1.3:  # Aspect ratio to check if sides are equal
                squares.append(approx)
                areas.append(width * height)

    squares = np.array(squares)
    areas = np.array(areas)
    indices = np.argsort(areas)
    squares = squares[indices[-64:]]
        
    # Draw the squares on the original image
    cv2.drawContours(frame, squares, -1, (0, 255, 0), 2)

    # Sort the squares by y-coordinate in descending order, then by x-coordinate in ascending order
    sorted_squares = sorted(squares, key=lambda x: (-np.mean(x[:, :, 1]), np.mean(x[:, :, 0])))

    # Function to get chessboard coordinates
    def get_chess_coord(index):
        row = index // 8
        col = index % 8
        return chr(97 + col) + str(1 + row)

    # Draw each square and annotate it
    for idx, square in enumerate(sorted_squares):
        # Draw the square (example: using bounding box)
        x, y, w, h = cv2.boundingRect(square)

        # Annotate with chessboard coordinates
        coord = get_chess_coord(idx)
        cv2.putText(frame, coord, (x + w//2, y + h//2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    test_red_clip.write(frame)

test_video.release()  # Release the video capture
test_red_clip.release()

Opened the file successfully.
File name: videos/EASY.mp4
Height: 1920.0, Width: 1080.0
FPS: 30.0


C:\Users\erykp\AppData\Local\Temp\ipykernel_18420\1207629656.py:106: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


KeyboardInterrupt: 

In [ ]:
!ffmpeg -hide_banner -loglevel error -i test_red_clip.avi -y test_clip.mp4

'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.
